In [1]:
import pandas as pd 
import os 

In [2]:
df = pd.read_csv('src/Sheet 1-Table 1.csv')

In [9]:
df['Josh R.'].notna().sum() / len(df['Josh R.'])

0.4744479495268139

In [13]:
df = df[['Date', 'Andrew', 'Josh R.']]
df.rename(columns = {'Date': 'date', 
    'Andrew':'andrew',
    'Josh R.': 'josh'}, inplace = True)

In [21]:
df['date'] = pd.to_datetime(df['date'], format = '%b %d, %Y')

In [23]:
df['josh'].str.replace(r'(\d+)h', r'\1 hours').str.replace(r'(\d+)m', r'\1 minutes').str.replace(r'(\d+)s', r'\1 seconds')

/var/folders/5f/_l7w33_x7rx9p0t5x7ly8zj80000gn/T/ipykernel_7948/3151614663.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['josh'].str.replace(r'(\d+)h', r'\1 hours').str.replace(r'(\d+)m', r'\1 minutes').str.replace(r'(\d+)s', r'\1 seconds')


0       2 minutes 28 seconds
1       2 minutes 33 seconds
2       2 minutes 19 seconds
3       3 minutes 44 seconds
4                        NaN
                ...         
1580              38 seconds
1581     3 minutes 3 seconds
1582    2 minutes 27 seconds
1583    1 minutes 14 seconds
1584              13 seconds
Name: josh, Length: 1585, dtype: object